In [126]:
import pandas as pd
from process_nerr_data import merge_wq_files, raw_nerr_data_path
from sqlalchemy import create_engine
wq_df = merge_wq_files(raw_nerr_data_path)

C:\Users\alexa\Documents\scripts\machine_learning\preprocess_data\process_nerr_data.py:143: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  (pd.read_csv(f, usecols=usecols, encoding=encoding) for f in files)


> c:\users\alexa\documents\scripts\machine_learning\preprocess_data\process_nerr_data.py(146)merge_files_by_dataset()
    144     df = strip_whitespace(df)
    145     df = convert_flags_to_numeric_code(df)
--> 146     return df
    147 
    148 

ipdb> continue


In [4]:
wq_df.head()

,StationCode,isSWMP,DateTimeStamp,Historical,ProvisionalPlus,F_Record,Temp,F_Temp,SpCond,F_SpCond,...,Level,F_Level,cLevel,F_cLevel,pH,F_pH,Turb,F_Turb,ChlFluor,F_ChlFluor
0,job06wq,P,12/06/1996 12:00,1,0,NaN,28.1,<4>,53.26,<4>,...,NaN,<-1>,NaN,NaN,8.3,<4>,NaN,<4>,NaN,<-1>
1,job06wq,P,12/06/1996 12:30,1,0,NaN,28.4,<4>,53.29,<4>,...,NaN,<-1>,NaN,NaN,8.3,<4>,NaN,<4>,NaN,<-1>
2,job06wq,P,12/06/1996 13:00,1,0,NaN,28.2,<4>,53.28,<4>,...,NaN,<-1>,NaN,NaN,8.2,<4>,NaN,<4>,NaN,<-1>
3,job06wq,P,12/06/1996 13:30,1,0,NaN,28.0,<4>,53.22,<4>,...,NaN,<-1>,NaN,NaN,8.1,<4>,NaN,<4>,NaN,<-1>
4,job06wq,P,12/06/1996 14:00,1,0,NaN,28.3,<4>,53.25,<4>,...,NaN,<-1>,NaN,NaN,8.2,<4>,NaN,<4>,NaN,<-1>


In [122]:
# Replaces original column series
# with an integer only series.
# Used with the apply df function.
def extract_flag_code(col):
    # One or more <, first group 
    # May have 1 or no -, followed by
    # any number of digits and capped by
    # a > or a not numeric.
    regex = "<?(-?\d+)>?\D?"
    integer_flags = col.str.extract(regex, expand=False)
    return integer_flags

# Select all flag columns...
flag_cols = df.filter(like='F_', axis=1).columns
flag_cols_df = df.loc[:, flag_cols]
# Need to convert columns dtypes to object because
# In cases where the entire column is nan pandas
# will assign a float dtype
flag_cols_df = flag_cols_df.astype("O")
# Replace non numeric values with numeric flag values
flag_cols_df = flag_cols_df.apply(lambda x: extract_flag_code(x))
# Convert to numeric
# This should NOT return an error if the regex did its job well
flag_cols_df = flag_cols_df.apply(lambda x: pd.to_numeric(x))
df.loc[:, flag_cols] = flag_cols_df
df

,StationCode,isSWMP,DateTimeStamp,Historical,ProvisionalPlus,F_Record,Temp,F_Temp,SpCond,F_SpCond,...,Level,F_Level,cLevel,F_cLevel,pH,F_pH,Turb,F_Turb,ChlFluor,F_ChlFluor
0,job06wq,P,12/06/1996 12:00,1,0,NaN,28.1,4,53.26,4,...,NaN,-1,NaN,NaN,8.3,NaN,NaN,4,NaN,-1
1,job06wq,P,12/06/1996 12:30,1,0,NaN,28.4,4,53.29,4,...,NaN,-1,NaN,NaN,8.3,NaN,NaN,4,NaN,-1
2,job06wq,P,12/06/1996 13:00,1,0,NaN,28.2,4,53.28,4,...,NaN,-1,NaN,NaN,8.2,NaN,NaN,4,NaN,-1
3,job06wq,P,12/06/1996 13:30,1,0,NaN,28.0,4,53.22,4,...,NaN,-1,NaN,NaN,8.1,NaN,NaN,4,NaN,-1
4,job06wq,P,12/06/1996 14:00,1,0,NaN,28.3,4,53.25,4,...,NaN,-1,NaN,NaN,8.2,NaN,NaN,4,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20768,job20wq,P,08/04/2020 8:00,0,0,NaN,NaN,-2,NaN,-2,...,NaN,-1,NaN,NaN,8.0,NaN,2.0,0,NaN,-1
20769,job20wq,P,08/04/2020 8:15,0,0,NaN,NaN,-2,NaN,-2,...,NaN,-1,NaN,NaN,8.0,NaN,2.0,0,NaN,-1
20770,job20wq,P,08/04/2020 8:30,0,0,NaN,NaN,-2,NaN,-2,...,NaN,-1,NaN,NaN,8.0,NaN,1.0,0,NaN,-1
20771,job20wq,P,08/04/2020 8:45,0,0,NaN,NaN,-2,NaN,-2,...,NaN,-1,NaN,NaN,8.0,NaN,1.0,0,NaN,-1


In [130]:
def create_db_engine():
    server = r"DESKTOP-4IL536V\SQLEXPRESS"
    database = r"machine_learning_pr"
    conn_str = f"mssql+pyodbc://{server}/{database}?driver=SQL+Server+Native+Client+11.0"
    engine = create_engine(conn_str)
    return engine

engine= create_db_engine()
df.to_sql("test", con=engine)

In [132]:
df.columns.tolist()

['StationCode',
 'isSWMP',
 'DateTimeStamp',
 'Historical',
 'ProvisionalPlus',
 'F_Record',
 'Temp',
 'F_Temp',
 'SpCond',
 'F_SpCond',
 'Sal',
 'F_Sal',
 'DO_Pct',
 'F_DO_Pct',
 'DO_mgl',
 'F_DO_mgl',
 'Depth',
 'F_Depth',
 'cDepth',
 'F_cDepth',
 'Level',
 'F_Level',
 'cLevel',
 'F_cLevel',
 'pH',
 'F_pH',
 'Turb',
 'F_Turb',
 'ChlFluor',
 'F_ChlFluor']